In [4]:
#import necessary packages
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from datetime import date
import warnings 

#from 2022_SummerInternship_FarallonInstitute.paul.Identify upwelling events (part 1) import compile_function

#Warnings
warnings.simplefilter('ignore') 

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from calendar import month_abbr
import glob


In [5]:
#Author: Paul
#compiles all voyages

#I've changed all instances of "sail to ds" and changed ddir to be a relative file path -will
def Compile_Datasets(fn_list_in):


    # fn_list_in: list of strings with the file names, or filename(string), or "all"
    # returns: compiled list
    fn_list = []
    
    ddir = "../saildrone_data"
    
    # Make sure the fn_list_in is formatted correctly
    if(fn_list_in == "all"):
        
        fn_list = glob.glob(ddir+ '/*.nc')
    elif(type(fn_list_in) == 'list' and type(fn_list_in[0]) == 'string'):
        fn_list = fn_list_in
    elif(type(fn_list_in) == 'string'):
        fn_list[0] = fn_list_in
    else: 
        raise Exception("first argument to 'Compile_Data_Set_And_Graph' function must be; a list of file names, a file name, or \"all\"")
        
    print()
    
    # open the first dataset
    sail = xr.open_dataset(fn_list[0])
    
    sail = sail.drop_vars("trajectory", errors='ignore')
    
    # give the first dataset a relative ID so all datasets can be differentiated
    sail["relativeID"] = 0
    # make lists for certain variables that remain constant for each dataset. these are used later in the last two cells
    yearList = [sail["time"][0].dt.year]
    durationList = [sail["time"][len(sail["time"]) - 1] - sail["time"][0]]
    # take the actual cruise ID from the dataset attributes and put it in a new list
    try:
        realID = [int(sail.attrs["id"])]
    except:
        realID = [fn_list[0]]
    sail["realID"] = realID[0]
    # add the duration back to the dataset
    sail["duration"] = durationList[0]

    # repeat previous steps for other datasets that need to be combined.

    if len(fn_list) > 1:
        for i in range(1, len(fn_list)):
            temp = xr.open_dataset(fn_list[i])
            temp = temp.drop_vars("trajectory", errors='ignore')
            temp["relativeID"] = i
            yearList.append(temp["time"][0].dt.year)
            
            try:
                realID.append(int(temp.attrs["id"]))
            except:
                realID.append(fn_list[i])
                
            tempDuration = temp["time"][len(temp["time"]) - 1] - temp["time"][0]
            temp["duration"] = tempDuration
            durationList.append(tempDuration)
            temp["realID"] = realID[i]
            sail = xr.concat([sail, temp], dim="time")
            temp.close()

    # reformat dates
    sail['date'] = mdates.date2num(sail['time'].dt.date)

    # ask what variable should be plotted
    return(sail)

In [6]:
#Author: William Gilmore
#Isolates data on the westcoast
def westcoast(upperLat, lowerLat):
    
    #upperLat: Upper acceptable latitude
    #lowerLat: Lower acceptable latitude
    # -function will discard all data not between upperLat and lowerLat
    
    # Runs Pauls function to gather all saildrone data into 1 dataset
    ds = Compile_Datasets("all")
    
    #Removes Data outside of Westcoast
    ds = ds.where((ds.lon > -130) & (ds.lon < -115) & (ds.lat > 27) & (ds.lat < 52))
    
    #Removes data that is not within 300 km of shore
    ds = ds.where(ds.dist_land <= 300)
    
    #Removes SF bay data
    ds = ds.where(~(((ds.lon > -122.5938) & (ds.lat > 37.72783)) & ((ds.lon < -122.2506620424831) & (ds.lat < 38.094658646550556))) | ~(((ds.lon > -122.38678630116495) & (ds.lat > 37.430464705762226)) & ((ds.lon < -121.99799777841487) & (ds.lat < 37.81408437558721))))
    
    #Removes Data not between upperLat and lowerLat
    ds = ds.where((ds.lat > lowerLat) & (ds.lat < upperLat))
     

    return(ds)

In [8]:
ds = westcoast(50, 20) # All westcoast voyages are within this range.
ds

<xarray.Dataset>
Dimensions:                       (time: 20913, date: 20913)
Coordinates:
  * time                          (time) datetime64[ns] 2019-07-22T04:11:00 ....
    ob                            (time) int64 0 1 2 3 4 ... 220 221 222 223 224
  * date                          (date) float64 1.81e+04 1.81e+04 ... 1.783e+04
Data variables: (12/44)
    lat                           (time) float64 42.79 42.66 ... 37.55 37.69
    lon                           (time) float64 -125.6 -125.6 ... -122.8 -122.6
    SOG_MEAN                      (time) float64 0.5036 0.408 ... 1.342 1.137
    COG_MEAN                      (time) float64 143.9 175.0 ... 54.54 92.8
    HDG_MEAN                      (time) float64 152.9 159.5 ... 49.98 87.61
    ROLL_MEAN                     (time) float64 1.004 0.7046 ... 1e+05 1e+05
    ...                            ...
    analysis_error                (time) float64 0.37 0.37 0.38 ... 0.38 0.39
    mask                          (time) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    sea_ice_fraction              (time) float64 nan nan nan nan ... nan nan nan
    relativeID                    (time) float64 0.0 0.0 0.0 ... 45.0 45.0 45.0
    realID                        (time) object 53453 53453 53453 ... 8708 8708
    duration                      (time) timedelta64[ns] 9 days 23:29:30 ... ...
Attributes: (12/45)
    area:                      Pacific Ocean
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    Conventions:               CF-1.6, ACDD-1.3, COARDS
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    ...                        ...
    time_coverage_end:         2019-08-02T19:59:00Z
    time_coverage_resolution:  PT1M
    time_coverage_start:       2019-06-18T00:00:00Z
    title:                     Saildrone West Coast Survey 2019
    Westernmost_Easting:       -126.2662016
    vehicle_id:                sd1043